# MongoDB Indexes
#### examples taken from http://img105.job1001.com/upload/adminnew/2015-04-07/1428393873-MHKX3LN.pdf

MongoDB offers two kinds of indexes:
<ol>
    Can be compound
    <li>Btree indexes</li>
    <ul>
        <li>Btree index</li>
        <li>Unique index</li>
    </ul>
    <li>Geospatial indexes</li> 
</ol>


## Creating and Mongo connection with pymongo driver (pymongo==3.7.2)

In [86]:
import sys
import pymongo
from pymongo import MongoClient
from pymongo.errors import ConnectionFailure

try:
    #Creating the connection client
    c = MongoClient(host="localhost", port=27017)
    #selecting the db
    myDb = c["test"]
    #Selection the collection (in sql = table)
    myCol = myDb["users"]
    #returning the connection obj
    print("Connected successfully to Mongo")
except (ConnectionFailure, e):
    sys.stderr.write("Could not connect to MongoDB: %s" % e)
    sys.exit(1)

Connected successfully to Mongo


## To create an index, we use the create_index method

The mandatory parameter is the property you want to be indexed
We can configure some tags in order to use specific behavior
<ul>
    <li><b>name="indexName":</b> To give a custom name, it will helps to drop the index by index name</li>
    <li><b>background=True:</b> To specify that an index should be built in the background </li>
    <li><b>unique=True:</b> To create an index with a unique constraint </li>
</ul>

In [87]:
#Document
user_doc = {
 "username":"goouser",
 "emails":[
 {
 "email":"goouser1@example.com",
 "primary":True
 },
 {
 "email":"goouser2@example2.com",
 "primary":False
 },
 {
 "email":"goouser3@example3.com",
 "primary":False
 }
 ]
}


In [88]:
#Document
user_doc2 = {
 "username":"foouser",
 "emails":[
 {
 "email":"foouser1@example.com",
 "primary":True
 },
 {
 "email":"foouser2@example2.com",
 "primary":False
 },
 {
 "email":"foouser3@example3.com",
 "primary":False
 }
 ]
}

In [89]:
#Deleting previous documents
myDb.users.drop()
#inserting the documento into MongoDB
myDb.users.insert_one(user_doc)
myDb.users.insert_one(user_doc2)

## Creating and index on username property

In [90]:
myDb.users.create_index("username", unique=True, name="username_idx" )

'username_idx'

## Showing Collection's indexes

In [8]:
myDb.users.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)], 'ns': 'test.users'},
 'username_idx': {'v': 2,
  'unique': True,
  'key': [('username', 1)],
  'ns': 'test.users'}}

## Retrieving all objects

In [9]:
#Printing pretty
from pprint import pprint

users = myDb.users.find()

for document in users: 
    pprint(document)

{'_id': ObjectId('5c794d3dbe25352d2ce6c81d'),
 'emails': [{'email': 'goouser1@example.com', 'primary': True},
            {'email': 'goouser2@example2.com', 'primary': False},
            {'email': 'goouser3@example3.com', 'primary': False}],
 'username': 'goouser'}
{'_id': ObjectId('5c794d3dbe25352d2ce6c81e'),
 'emails': [{'email': 'foouser1@example.com', 'primary': True},
            {'email': 'foouser2@example2.com', 'primary': False},
            {'email': 'foouser3@example3.com', 'primary': False}],
 'username': 'foouser'}


## Retrieving one element

In [10]:
user = myDb.users.find_one({"username" : "goouser"})

pprint(user)

{'_id': ObjectId('5c794d3dbe25352d2ce6c81d'),
 'emails': [{'email': 'goouser1@example.com', 'primary': True},
            {'email': 'goouser2@example2.com', 'primary': False},
            {'email': 'goouser3@example3.com', 'primary': False}],
 'username': 'goouser'}


## Query explain

In [11]:
myDb.users.find({"username" : "goouser"}).explain() ; # remove ; to see the result

## Compound indexes
A list of 2-tuples (key, direction) 

In [12]:
# Create a compound index called "name_idx" on first_name and last_name properties
# with ascending index direction

myDb.users.create_index([("first_name", pymongo.ASCENDING),("last_name", pymongo.ASCENDING)], name="name_idx")

'name_idx'

 ## GeoSpatial Indexing
 MongoDB provides the *near* and *within*  
*near*: proximity to a given point. 
*within*: allows you to specify a bounds for the query.  
Supported boundary definitions include.  
*box* for a rectangular shape.  
*circle* for a circle.  
*polygon* operator allows for convex and concave polygon boundaries.  

## Inserting a document with coordinates

In [73]:
import bson
# location property is a sub-document with x,y ordering
loc = bson.SON()
loc["x"] = 5
loc["y"] = 5
user_doc3 = {
 "username":"hoouser",
 "user_location":loc
}
loc2 = bson.SON()
loc2["x"] = 100
loc2["y"] = 100
user_doc4 = {
 "username":"moouser",
 "user_location":loc2
}

#Deleting previous documents
myDb.usersLocation.drop()

myDb.usersLocation.insert_one(user_doc4);
myDb.usersLocation.insert_one(user_doc3);

#Note that we create a new collection called usersLocation

In [46]:
for document in myDb.usersLocation.find(): 
    pprint(document)

{'_id': ObjectId('5c79586cbe25352d2ce6c82a'),
 'user_location': {'x': 100, 'y': 100},
 'username': 'moouser'}
{'_id': ObjectId('5c79586cbe25352d2ce6c82b'),
 'user_location': {'x': 5, 'y': 5},
 'username': 'hoouser'}


## Creating the index

In [48]:
# Create geospatial index on "user_location" property.
myDb.usersLocation.create_index([("user_location", pymongo.GEO2D), ("username",
pymongo.ASCENDING)],name="location_idx")

'location_idx'

## Making a query within box
To specify a rectangle to search within, you simply provide the lower-left and top-right
co-ordinates as elements in an array

In [50]:
box = [[-10, -10], [20, 20]]
users_in_boundary = myDb.usersLocation.find({"user_location":{"$within": {"$box":box}}})
for document in users_in_boundary: 
    pprint(document)

{'_id': ObjectId('5c79586cbe25352d2ce6c82b'),
 'user_location': {'x': 5, 'y': 5},
 'username': 'hoouser'}


## Making a query within circle
To specify a citcle to search within, you just supply the center point and the radius(degrees)


In [82]:
users_in_circle = myDb.usersLocation.find({"user_location":{"$within":{"$center":[[100, 100],5]}}}).limit(10)
for document in users_in_circle: 
    pprint(document)

{'_id': ObjectId('5c795fbdbe25352d2ce6c82c'),
 'user_location': {'x': 100, 'y': 100},
 'username': 'moouser'}


## The spherical model
can be used by employing the nearSphere and circle
Sphere variants on the near and circle operators  
The are always expressed in radians  
To translate from kilometers to radians, simply divide the kilometer value by the radius of the earth which is approximately 6371 km (or 3959 miles)  

In [85]:
earth_radius_km = 6371.0
max_distance_km = 35.0
max_distance_radians = max_distance_km / earth_radius_km
nearest_users = myDb.usersLocation.find( {"user_location": 
                                          {"$geoWithin" : { "$centerSphere": [ [ 5,5 ], max_distance_km ] }}})
for document in nearest_users: 
    pprint(document)

{'_id': ObjectId('5c795fbdbe25352d2ce6c82d'),
 'user_location': {'x': 5, 'y': 5},
 'username': 'hoouser'}
